In [13]:
import pandas as pd
import numpy as np
import time
import pickle
pd.set_option('max_colwidth', None)
pd.set_option("max_columns", None)

In [14]:
train_df = pd.read_csv("../datasets/SDP_train.csv")
test_df  = pd.read_csv("../datasets/SDP_test.csv")

In [15]:
train_df.head(5)

,unique_id,core_id,citing_title,citing_author,cited_title,cited_author,citation_context,citation_influence_label
0,CC1,158977742,Ontology-Based Recommendation of Editorial Products,Thiviyan Thanapalasingam,Web search personalization with ontological user profiles,Sieg,They usually generate user models that describe user interests according to a set of features #AUTHOR_TAG,0
1,CC2,158977742,Ontology-Based Recommendation of Editorial Products,Thiviyan Thanapalasingam,Exploring Scholarly Data with Rexplore,Osborne,The Computer Science Ontology (CSO)[3]is a large-scale and granular ontology of research topics that was created automatically by running the Klink-2 algorithm [1]on the Rexplore dataset #AUTHOR_TAG.,0
2,CC3,158977742,Ontology-Based Recommendation of Editorial Products,Thiviyan Thanapalasingam,Klink-2: Integrating Multiple Web Sources to Generate Semantic Topic Networks,Osborne,"In order to do so, we characterized all SN publications according to their associated research topics by exploiting the Computer Science Ontology (CSO), a large-scale automatically generated taxonomy of research areas #AUTHOR_TAG",0
3,CC4,158977742,Ontology-Based Recommendation of Editorial Products,Thiviyan Thanapalasingam,Forecasting the Spreading of Technologies in Research Communities,Osborne,"This API supports a number of applications, including Smart Book Recommender, Smart Topic Miner [5], the Technology-Topic Framework #AUTHOR_TAG, a system that forecasts the propagation of technologies across research communities, and the Pragmatic Ontology Evolution Framework [7], an approach to ontology evolution that is able to select new concepts on the basis of their contribution to specific computational tasks",1
4,CC5,158977742,Ontology-Based Recommendation of Editorial Products,Thiviyan Thanapalasingam,Supporting Springer Nature Editors by means of Semantic Technologies,Osborne,"It works according to three main steps:1) It represents journals, books, and conferences according to the metadata of their chapters/articles and uses the Smart Topic API #AUTHOR_TAG to characterize each of them with a semantically enhanced topic vector",1


In [16]:
#train_df['cited_text_all'] = train_df['citing_title'] + '.' +  train_df['cited_title'] + '.' + train_df['citation_context']
#test_df['cited_text_all'] = test_df['citing_title'] + '.' + test_df['cited_title'] + '.' + test_df['citation_context']


train_df['cited_text_all'] =  train_df['cited_title'] + '.' + train_df['citation_context']
test_df['cited_text_all'] =  test_df['cited_title'] + '.' + test_df['citation_context']

In [17]:
train_df['cited_text_all'].head(2)

0                                                                               Web search personalization with ontological user profiles.They usually generate user models that describe user interests according to a set of features #AUTHOR_TAG
1    Exploring Scholarly Data with Rexplore.The Computer Science Ontology (CSO)[3]is a large-scale and granular ontology of research topics that was created automatically by running the Klink-2 algorithm [1]on the Rexplore dataset #AUTHOR_TAG.
Name: cited_text_all, dtype: object

In [18]:
s='They usually generate user models that describe user interests according to a set of features #AUTHOR_TAG'
s.replace('#AUTHOR_TAG', ' ')

'They usually generate user models that describe user interests according to a set of features  '

### Preprocessing

In [19]:
#import nltk
#nltk.download('wordnet')

In [20]:
from preprocessor import Preprocessing
preprocessed_data = []
pre = Preprocessing()

train_df['cited_text_all_cleaned'] = train_df['cited_text_all'].apply(lambda x:pre.preprocess(x))
train_df['cited_text_all_cleaned'] = train_df['cited_text_all_cleaned'].str.lower()

In [21]:
test_df['cited_text_all_cleaned'] = test_df['cited_text_all'].apply(lambda x:pre.preprocess(x))
test_df['cited_text_all_cleaned'] = test_df['cited_text_all_cleaned'].str.lower()

In [22]:
test_df['cited_text_all_cleaned'].head(2)

0    the english village community examined relation manorial tribal system common open field system husbandry essay economic history.in historical research , regressive method moving step-by-step back time better known later situation already applied bloch reconstruction medieval agrarian landscape france used since within historical research
1                                                                                                                                        die italienischen begr nder der wiener donaukartographie.for example , new cut-off channel donaukanal excavated 1700-1703 bridge built 1704 later added , could serve basis proposed hydraulic construction 1712
Name: cited_text_all_cleaned, dtype: object

### Build embeddings

In [23]:
from sentence_transformers import SentenceTransformer

In [24]:
MODELNAME = 'all-mpnet-base-v2'
# 'quora-distilbert-base' time to build emb 225.0789451599121 seconds
# 'all-mpnet-base-v2'     Time to build emb 482.
model = SentenceTransformer(MODELNAME)


In [25]:
# print("Building embeddings")
# start = time.time()
# encoded_data = model.encode(train_df['cited_text_all'].tolist())
# encoded_data = np.asarray(encoded_data.astype('float32'))
# end = time.time()
# print("Time to build emb {} seconds".format(end-start))

In [26]:
colnames = ["dim_" + str(i) for i in range(0,768)]
print(colnames[0:10])

['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_6', 'dim_7', 'dim_8', 'dim_9']


In [27]:
test_df.shape

(1000, 9)

In [28]:
train_df['cited_emb_all'] = train_df['cited_text_all'].apply(lambda x:model.encode(x)) 
emb_df = pd.DataFrame(train_df['cited_emb_all'].to_list(), columns = colnames)
train_df 2= pd.concat([train_df,emb_df],axis=1)
train_df.to_pickle('../datasets/SDP_train_and_emb_mpnet.pkl')

In [29]:
test_df['cited_emb_all'] = test_df['cited_text_all'].apply(lambda x:model.encode(x)) 
emb_df = pd.DataFrame(test_df['cited_emb_all'].to_list(), columns = colnames)
test_df = pd.concat([test_df,emb_df],axis=1)
test_df.to_pickle('../datasets/SDP_test_and_emb_mpnet.pkl')

In [30]:
train_df.shape

(3000, 779)